In [17]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

import os
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser, TrainingArguments, pipeline,logging)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer




In [22]:
!pip install huggingface_hub

from langchain_huggingface import HuggingFaceEndpoint
from huggingface_hub import login

from google.colab import userdata
sec_key = userdata.get("HF_tkn")
os.environ["HF_tkn"] = sec_key

In [33]:
model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"  #"mistralai/Mistral-7B-v0.3"

dataset_name = "stas/openwebtext-10k"


lora_r = 32  #  reducing from 64
lora_alpha = 16
lora_dropout = 0.1

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

output_dir = "./results"
num_train_epochs = 3
fp16 = True
bf16 = False
per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 2
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

max_seq_length = 256
packing = False
#device_map = {"": 0}
device_map = "cuda" if torch.cuda.is_available() else "cpu"


########################


dataset = load_dataset(dataset_name, split="train[:4000]")  # Loads 4000 samples


compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

#if compute_dtype == torch.float16 and use_4bit:
#    major, _ = torch.cuda.get_device_capability()
#    if major >= 8:
#        print("=" * 80)
#        print("Your GPU supports bfloat16: accelerate training with bf16=True")
#        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


target_modules = [
    "self_attention.query_key_value",
    "self_attention.dense",
    "mlp.dense_h_to_4h",
    "mlp.dense_4h_to_h"
]


training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)


trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="instruction",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train()

README.md:   0%|          | 0.00/951 [00:00<?, ?B/s]

openwebtext-10k.py:   0%|          | 0.00/3.08k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/30.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at HuggingFaceTB/SmolLM2-1.7B-Instruct and are newly initialized: ['model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'lm_head.weight', 'model.layers.9.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.5.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.lay

ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.

LocalTokenNotFoundError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs

logging.set_verbosity(logging.CRITICAL)

In [ ]:
def generate_text(prompt, model, tokenizer, max_length):
    instruction = f"[INST] {prompt} [/INST]"
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length)
    result = pipe(instruction)
    return result[0]['generated_text']

In [ ]:
prompt = "Explain me the steps involved in photosynthesis"
generated_text = generate_text(prompt, model, tokenizer, 250)
print(generated_text)

In [ ]:
prompt = "Can you present a flow chart of decision making"
generated_text = generate_text(prompt, model, tokenizer, 500)
print(generated_text)

In [ ]:
print(generate_text("What is the relation between economy and inflation?", model, tokenizer, 500))